In [1]:
import numpy as np
import pandas as pd
import config

In [2]:
df = pd.read_csv(config.CONFIG['paths']['train_with_folds'])
print('Shape', df.shape)
df.head()

Shape (1460, 290)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,fold
0,1,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2
1,2,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4
2,3,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2
3,4,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1
4,5,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3


# Декомпазиция

In [3]:
df_fe = df.copy()

if 'YearBuilt' in df_fe.columns and 'YrSold' in df_fe.columns:
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']

if 'YearRemodAdd' in df_fe.columns and 'YrSold' in df_fe.columns:
    df_fe['RemodAge'] = df_fe['YrSold'] - df_fe['YearRemodAdd']

# Комбинация

In [4]:
if 'GrLivArea' in df_fe.columns and 'TotalBsmtSF' in df_fe.columns:
    df_fe['TotalSF'] = df_fe['GrLivArea'] + df_fe['TotalBsmtSF']
elif 'GrLivArea' in df_fe.columns:
    df_fe['TotalSF'] = df_fe['GrLivArea']

if 'FullBath' in df_fe.columns and 'HalfBath' in df_fe.columns:
    df_fe['TotalBath'] = df_fe['FullBath'] + 0.5 * df_fe['HalfBath']
    if 'BsmtFullBath' in df_fe.columns:
        df_fe['TotalBath'] = df_fe['TotalBath'] + df_fe['BsmtFullBath']
    if 'BsmtHalfBath' in df_fe.columns:
        df_fe['TotalBath'] = df_fe['TotalBath'] + 0.5 * df_fe['BsmtHalfBath']

porch_cols = [c for c in ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'] if c in df_fe.columns]
if porch_cols:
    df_fe['TotalPorch'] = df_fe[porch_cols].sum(axis=1)

# Трансформации

In [5]:
if 'LotArea' in df_fe.columns:
    df_fe['LotArea_log'] = np.log1p(df_fe['LotArea'])

if 'GrLivArea' in df_fe.columns and 'TotRmsAbvGrd' in df_fe.columns:
    df_fe['LivAreaPerRoom'] = df_fe['GrLivArea'] / df_fe['TotRmsAbvGrd'].replace(0, np.nan)

if 'OverallQual' in df_fe.columns and 'GrLivArea' in df_fe.columns:
    df_fe['Qual_x_Area'] = df_fe['OverallQual'] * df_fe['GrLivArea']

In [6]:
for col in df_fe.columns:
    if df_fe[col].dtype in [np.float64, np.float32]:
        df_fe[col] = df_fe[col].replace([np.inf, -np.inf], np.nan)
        if df_fe[col].isna().any():
            df_fe[col] = df_fe[col].fillna(df_fe[col].median())
print('Shape после FE:', df_fe.shape)
df_fe.head()

Shape после FE: (1460, 298)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleCondition_Partial,fold,HouseAge,RemodAge,TotalSF,TotalBath,TotalPorch,LotArea_log,LivAreaPerRoom,Qual_x_Area
0,1,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,...,0.0,2,-0.912216,-0.739891,-0.088969,2.390813,-1.281545,-0.232111,0.405974,0.241265
1,2,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,...,0.0,4,-0.771172,-0.184862,-0.016047,1.563370,0.175839,-0.096386,1.514080,0.034662
2,3,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,...,0.0,2,-0.845975,-0.691437,0.201644,2.390813,-1.568410,0.070906,-1.616064,0.335520
3,4,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,...,0.0,1,0.495977,-0.647357,-0.303665,-0.419572,2.777752,-0.101919,1.292812,0.249946
4,5,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,...,0.0,3,-0.812854,-0.594530,1.499005,2.390813,0.598085,0.318562,0.850535,1.786306


In [7]:
path_fe = config.CONFIG['paths']['train_with_folds_fe']
df_fe.to_csv(path_fe, index=False)
print('Сохранено:', path_fe)

Сохранено: C:\newTry2\classicMLpractice\ProjectKaggle\HousePrices\checkpoints\train_with_folds_fe.csv
